# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings("ignore")

# Load Data

In [ ]:
movie_df = pd.read_csv("/mnt/d/Datasets/movies_metadata.csv", usecols=["id", "title"])
movie_df.columns = ["movieId", "title"]
movie_df.head()

In [ ]:
movie_df.shape

In [ ]:
movie_df = movie_df[~movie_df.movieId.str.contains("-")]

In [ ]:
movie_df["movieId"] = movie_df["movieId"].astype("int")

In [ ]:
rating_df = pd.read_csv("/mnt/d/Datasets/ratings.csv", usecols=["userId", "movieId", "rating"])
rating_df.head()

In [ ]:
rating_df.shape

# Merge 

In [ ]:
combined_df = pd.merge(rating_df, movie_df, on='movieId')
combined_df = combined_df.drop_duplicates(["userId", "title"])
combined_df = combined_df.dropna(subset=['title'])

In [ ]:
combined_df.head()

# Matrix

In [ ]:
user_rating_matrix = combined_df.pivot(index='userId', columns='title', values='rating').fillna(0)

In [ ]:
user_rating_matrix.head()

# Correlation

In [ ]:
svd = TruncatedSVD(n_components=10, random_state=42)

In [ ]:
transformed_matrix = svd.fit_transform(user_rating_matrix.values.T)

In [ ]:
correlation_matrix = np.corrcoef(transformed_matrix)

In [ ]:
movie_list = list(user_rating_matrix.columns)

# Recommend

In [52]:
def recommend_movies(correlation_matrix, movie_list, target_movie, threshold=0.9, count=25):
    if target_movie not in movie_list:
        raise ValueError(f"Movie '{target_movie}' not found in the dataset.")
        
    movie_index = movie_list.index(target_movie)
    similar_movies = correlation_matrix[movie_index]
    return [movie for movie, corr in zip(movie_list, similar_movies) if corr >= threshold][:count]

In [ ]:
target_movie = "Toy Story"
recommended_movies = recommend_movies(correlation_matrix, movie_list, target_movie)
print("Recommended movies:")
for i, recommended in enumerate(recommended_movies):
    print(f"{i} - {recommended}")